In [1]:
import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
import os
import re
from encode_utils.new_mask_utils import randomsingle, useall
from encode_utils.eval_utils import all_lattice_multi, mean, all_unnoun_multi
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

2022-12-25 10:29:44.358834: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-25 10:29:44.358859: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# run pipeline for generating lattice preds, using different parameters

In [7]:
# set up information for set
col = {
    "noun_xsum": ["nounsum_reversed/", "nounxsumlargeexplodev2.csv"],
    "noun_fren": ["frtest_reversed/", "nounlargeexplodev1.csv"],
    "mt_fren": ["frtest_reversed/", "frenchlargeexplodev1.csv"],
    "mt_ende": ["detest_reversed/", "germanlargeexplodev1.csv"],
    "mt_enru": ["rutest_reversed/", "russianlargeexplodev1.csv"],
}
curcol = "mt_fren"
gsuffix = col[curcol][0]
expl_fname = col[curcol][1]
base = "outputs/graph_pickles/"+gsuffix
goldmetric = "dupcqe"
explode_df = pd.read_csv("outputs/score_csvs/"+expl_fname)
#TODO switch back for french-english
if "fren" in curcol:
    SETLEN = len(os.listdir(base))
else:
    SETLEN = int(len(os.listdir(base))/2)

In [4]:
# use noun model
if "noun" in expl_fname:
    encodemod = get_effrerank_model("noun")
# use mt model (causal)
else:
    encodemod = get_effrerank_model("comstyle")
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_43/checkpoints/epoch=3-step=130000.ckpt.


freeze embeds


In [8]:
args = {
    'setlen':int(SETLEN),
    'tok':xlm_tok, 
    'dev':device,
    'model':encodemod,
    'explode_df':explode_df,
    'base':base,
    'goldmetric':goldmetric,
    'device':device, 
    'efficient':False,
    'noregen':False
}

In [9]:
latpreds = all_lattice_multi(32, default_scofunct, randomsingle, args)

417   418   tensor(0.9264, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.8860, device='cuda:2', grad_fn=<SumBackward0>)  
418   419   tensor(0.8060, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.7811, device='cuda:2', grad_fn=<SumBackward0>)  
419   420   tensor(0.6336, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.6336, device='cuda:2', grad_fn=<SumBackward0>)  
420   421   tensor(0.7921, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.7990, device='cuda:2', grad_fn=<SumBackward0>)  
421   422   tensor(0.7295, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.7295, device='cuda:2', grad_fn=<SumBackward0>)  
422   423   tensor(0.7490, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.7533, device='cuda:2', grad_fn=<SumBackward0>)  
423   424   tensor(0.9312, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.9312, device='cuda:2', grad_fn=<SumBackward0>)  
424   425   tensor(0.7528, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.7528, device='cuda:2', gr

In [ ]:
latpreds

In [ ]:
graph = pickle.load(open(base+str(0), 'rb'))

In [27]:
graph['root'].nextlist[1].token_str

'Two'

In [ ]:
graph[3]

In [16]:
(len(explode_df)/1000)/(5*32)

4.2108125

In [10]:
print(mean(latpreds['goldsco']))
print(mean(latpreds['hypsco']))
print(mean(latpreds['goldsco'])-mean(latpreds['hypsco']))

tensor(0.7730, device='cuda:2', grad_fn=<DivBackward0>)
tensor(0.7619, device='cuda:2', grad_fn=<DivBackward0>)
tensor(0.0112, device='cuda:2', grad_fn=<SubBackward0>)


In [11]:
gs = list(latpreds['hypsco'])
hs = list(latpreds['goldsco'])

In [12]:
mean([max(gs[i], hs[i]) for i in range(len(latpreds))])

tensor(0.7815, device='cuda:2', grad_fn=<DivBackward0>)

In [13]:
outfile = "mt_comstyle_fren_randsing_32shot.csv"
latpreds.to_csv("outputs/predcsvs/"+outfile)

In [13]:
latpreds[latpreds['numnodes']<600]

,hyp,hypsco,gold,goldsco,ascos,ahyps,numnodes,src,ref
0,"Liverpool City Council has said it is "" job do...","tensor(1.0552, device='cuda:2', grad_fn=<SumBa...",Liverpool City Council's decision to back Ever...,"tensor(1.1769, device='cuda:2', grad_fn=<SumBa...",[1.05520761013031],"[Liverpool City Council has said it is "" job d...",472,Liverpool Mayor Joe Anderson said it was a gre...,Everton's ambition to build a new Â£300m stadi...
1,"Tata Steel, the UK's biggest steel maker, was ...","tensor(1.2895, device='cuda:2', grad_fn=<SumBa...","Tata Steel, the UK's largest steelmaker, made ...","tensor(1.1844, device='cuda:2', grad_fn=<SumBa...",[1.2895307540893555],"[Tata Steel, the UK's biggest steel maker, was...",381,Three separate experts say Tata made the cash ...,Tata Steel is refusing to comment on claims it...
2,Sting is to be given a lifetime achievement aw...,"tensor(1.1859, device='cuda:2', grad_fn=<SumBa...",Sting will be honoured with a lifetime achieve...,"tensor(1.2032, device='cuda:2', grad_fn=<SumBa...",[1.185947060585022],[Sting is to be given a lifetime achievement a...,547,The star will be honoured for his contribution...,"Sting is to join the likes of Ray Davies, Van ..."
4,A book about a man who is left brain dead afte...,"tensor(1.2818, device='cuda:2', grad_fn=<SumBa...",A book about a man who is left brain dead afte...,"tensor(1.2792, device='cuda:2', grad_fn=<SumBa...",[1.2818293571472168],[A book about a man who is left brain dead aft...,467,Maylis de Kerangal is the first French author ...,"Mend the Living, a ""heart-breaking"" novel abou..."
5,Tributes have been paid to former Labour cabin...,"tensor(1.0479, device='cuda:2', grad_fn=<SumBa...","Tributes have been paid to Lord Healey, the fo...","tensor(1.0062, device='cuda:2', grad_fn=<SumBa...",[1.0478570461273193],[Tributes have been paid to former Labour cabi...,564,Lord Healey was Labour's defence secretary fro...,Former chancellor Denis Healey has died at the...
...,...,...,...,...,...,...,...,...,...
507,Police investigating the disappearance of a Me...,"tensor(1.1342, device='cuda:2', grad_fn=<SumBa...",Police investigating the disappearance of a Me...,"tensor(0.9882, device='cuda:2', grad_fn=<SumBa...",[1.1342251300811768],[Police investigating the disappearance of a M...,424,Steven Preston went missing on 18 September 19...,New information suggests a man who disappeared...
508,World number one Angel ique Ker ber beat six -...,"tensor(1.5848, device='cuda:2', grad_fn=<SumBa...",World number one Angelique Kerber beat six-tim...,"tensor(1.6015, device='cuda:2', grad_fn=<SumBa...",[1.584815502166748],[World number one Angel ique Ker ber beat six ...,470,Media playback is not supported on this device...,Germany's Angelique Kerber stunned world numbe...
509,The BBC is to publish the salaries of its top ...,"tensor(1.1270, device='cuda:2', grad_fn=<SumBa...",The BBC is to publish the salaries of its top ...,"tensor(1.1711, device='cuda:2', grad_fn=<SumBa...",[1.1269809007644653],[The BBC is to publish the salaries of its top...,466,It's the first time - and probably the only ti...,The salaries of the BBC's biggest stars are ex...
510,Canadian plane manufacturer Bomb ardier has sa...,"tensor(1.1305, device='cuda:2', grad_fn=<SumBa...",Canadian plane manufacturer Bombardier has rev...,"tensor(1.0996, device='cuda:2', grad_fn=<SumBa...",[1.130475401878357],[Canadian plane manufacturer Bomb ardier has s...,491,The company's decision last week to grant the ...,Canadian plane and train manufacturer Bombardi...


In [7]:
import time

In [ ]:
s = time.time()
args['noregen'] = True
args['setlen'] = 100
phoc = all_unnoun_multi(32, "utnoun", args)
print(mean(phoc))
e = time.time()

In [11]:
(e-s)/len(phoc)

0.9002537321537099

In [ ]:
args['explode_df']

In [37]:
explode_df['phocrank'] = explode_df['posthoc']*-1
#rand = all_unnoun_multi(32)

In [32]:
mean(rand)

tensor(0.8749, device='cuda:0', grad_fn=<DivBackward0>)

In [49]:
latpreds[latpreds['goldsco']-latpreds['hypsco']>.2]

,hyp,hypsco,gold,goldsco,ascos,ahyps,numnodes
7,Немецкие и французские стороны явно нуждались ...,"tensor(0.3987, device='cuda:0', grad_fn=<SumBa...",Германия и Франция явно нуждались во взаимном ...,"tensor(0.9256, device='cuda:0', grad_fn=<SumBa...",[0.3987096846103668],[Немецкие и французские стороны явно нуждались...,385
13,Небольшая часть населения Индии не имеет ни од...,"tensor(0.6347, device='cuda:0', grad_fn=<SumBa...",Сложно понять низкую позицию Индии.,"tensor(0.9250, device='cuda:0', grad_fn=<SumBa...",[0.6347033977508545],[Небольшая часть населения Индии не имеет ни о...,501
19,Так много о предсказках Кругмана.,"tensor(0.2668, device='cuda:0', grad_fn=<SumBa...","Это все, что предсказывал Кругман.","tensor(0.7456, device='cuda:0', grad_fn=<SumBa...",[0.2667738199234009],[Так много о предсказках Кругмана.],554
31,"Мы узнали, что это была просто воздушная пузыр...","tensor(0.8288, device='cuda:0', grad_fn=<SumBa...","Мы поняли, что это было просто кружево.","tensor(1.0661, device='cuda:0', grad_fn=<SumBa...",[0.8288499116897583],"[Мы узнали, что это была просто воздушная пузы...",558
71,В некоторых странах до 30% более оптимистичных...,"tensor(0.7622, device='cuda:0', grad_fn=<SumBa...",До 30% наиболее оптимистичных потребителей в н...,"tensor(1.0325, device='cuda:0', grad_fn=<SumBa...",[0.7622228264808655],[В некоторых странах до 30% более оптимистичны...,381
